In [ ]:
!pip install requests-cache

In [5]:
import requests_cache
import yfinance as yf

session = requests_cache.CachedSession(backend='memory')

df=yf.download("GOOGL", period="1y", interval="1d")

df


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-16,102.070000,103.129997,100.940002,102.800003,102.800003,42797200
2022-09-19,101.750000,103.330002,101.550003,103.070000,103.070000,23036800
2022-09-20,102.080002,102.370003,100.519997,101.139999,101.139999,26517100
2022-09-21,101.669998,102.879997,99.269997,99.279999,99.279999,31487300
2022-09-22,98.820000,101.239998,98.790001,100.139999,100.139999,34298600
...,...,...,...,...,...,...
2023-09-11,136.539993,137.479996,135.789993,136.919998,136.919998,20763400
2023-09-12,136.259995,136.869995,135.190002,135.339996,135.339996,18405500
2023-09-13,135.089996,136.899994,134.149994,136.710007,136.710007,20749500
